# Phenotyping of Fly Wings

## 1. Open the Dataset and Browse the Available Features

In the folder "data" you will find the dataset used in the paper [Assessing the potential of vision language models for automated phenotyping of Drosophila melanogaster](https://www.biorxiv.org/content/biorxiv/early/2024/05/27/2024.05.27.594652.full.pdf). The dataset is composed of a XLSX file and an additional "data" folder containing the images associated with each fly wing. The first task is to:
1. open the dataset and explore it
2. examine whether any information is missing
3. using the paper, study how the dataset was created
4. find a way to open the dataset using Python

In [2]:
# tip: feel free to use an AI tool for learning how to open the dataset, but you should be able to explain what the tool does

# 2. Perform a Classification

By examining the dataset you can select a feature you want to predict based on the image. It could be, as in the paper linked above, the aberrant vs wildtype phenotye or it could be predicting generally the type of phenotype or it could be something else of your own choice. Compare the performance of two of the classified discussed in class. Be careful, in this setting you don't have many training data - are there other evaluation practices that can help you assessing performance apart from train/test split? 

In [ ]:
# tip: you need two things: 1) to be able to open in Python the images and 2) to select a label from the spreadsheet

# 3. Examine the Errors the Tools Make

Compare the errors the classifiers make (which images are mis-labelled?). Do you notice any specific type of pattern? How good are performance overall? Would you use a tool like this as part of an experimental pipeline? What would be the trade-offs?

In [3]:
# tip: you need to associate each classified image with the predicted label. Then you can manually examine a few of the errors

# 4. Optional: LLMs for Image Classification

In the paper linked above, we used a LLM for image classification. You can try the same and then discuss what do you think are the major differences compared to training a machine learning classifier. Which benefits do you see? What are the drawbacks?

Feel free to use any LLM you want. You don't need to redo the entire classification, a specific experiment to compare performance will be good enough.